In [1]:
import pandas as pd #판다스
import numpy as np #넘파이
import seaborn as sns #씨본
import re #정규식
import matplotlib
import matplotlib.pyplot as plt #시각화
%matplotlib inline

In [2]:
from geoband.API import *
import pandas as pd

GetCompasData('SBJ_2208_001', '10', '10_연수구_건축물대장_표제부.xlsx')

[2022-09-30 16:42:44] 성공 - 10_연수구_건축물대장_표제부.xlsx


'10_연수구_건축물대장_표제부.xlsx'

In [3]:
# 건축물대장 표제부 : Building
# 2022년 기준 데이터

Building = pd.read_excel('10_연수구_건축물대장_표제부.xlsx')
Building.head()

,local_location,sgg_cd,emd_cd,land_gbn,bun,ji,mgm_bldrgst_pk,regstr_gb_cd_nm,bldg_nm,dong_nm,etc_purpose_nm,hshld_num,use_apr_day
0,인천광역시 연수구 옥련동 105번지,28185,10100,0,105,0,28185-604,일반,NaN,NaN,주택,0,NaN
1,인천광역시 연수구 옥련동 112-6번지,28185,10100,0,112,6,28185-607,일반,NaN,NaN,단독주택,1,19720803
2,인천광역시 연수구 옥련동 115-12번지,28185,10100,0,115,12,28185-100179967,일반,NaN,NaN,"단독주택, 제1,2종근린생활시설",0,20091014
3,인천광역시 연수구 옥련동 116-10번지,28185,10100,0,116,10,28185-609,일반,NaN,NaN,단독주택,0,19850202
4,인천광역시 연수구 옥련동 116-11번지,28185,10100,0,116,11,28185-100180087,일반,NaN,NaN,다가구주택,0,20091109


# 전처리

In [4]:
Building2 = Building.copy()

In [5]:
# 불필요 칼럼 삭제

# sgg_cd (시군구코드)
# land_gbn (대지구분코드)
# emd_cd (읍면동코드)
# bun (번)
# ji (지)
# mgm_bldrgst_pk (건축물대장 pk)
# bldg_nm (건물 명칭)
# dong_nm (동 명칭)
# hshld_num (세대수)

Building2 = Building2.drop(columns = ['sgg_cd', 'land_gbn', 'emd_cd', 'bun', 'ji', 'mgm_bldrgst_pk', 
                                                                          'bldg_nm', 'dong_nm', 'hshld_num'], axis = 1)

In [6]:
Building2.columns

Index(['local_location', 'regstr_gb_cd_nm', 'etc_purpose_nm', 'use_apr_day'], dtype='object')

In [7]:
# 사용승인 일자 null 값 삭제

Building2 = Building2.dropna(axis = 0)

In [8]:
# local_location : 대지위치

# null 값 확인
print('null :', Building2['local_location'].isnull().sum())

# 중복을 제외하고, 모두 다른 위치 !
# 중복의 경우, ㅇㅇ아파트 1동 등으로 보임
print('unique len :',len(Building2['local_location'].unique()))

null : 0
unique len : 5953


In [9]:
# '동' 추출하기

# 빈 리스트 생성 > 동이름 저장
temp2 = []
dong = []

# 동만 추출하여 dong 리스트에 저장
for i in Building2['local_location']:
    temp2 = i.split(' ')
    # print(temp2)
    dong.append(temp2[2])
    
print(len(dong)) #확인용
# print(dong) #확인용

8915


In [10]:
# 대지 위치 값을 "동"만 있는 값으로 대체
Building2['local_location'] = dong

In [11]:
# 칼럼명 변경 (local_location > emd_nm)
Building2 = Building2.rename(columns = {'local_location' : 'emd_nm'})

In [12]:
Building2.reset_index(inplace = True)
Building2.drop('index', axis = 1, inplace =True)

In [13]:
## 연도만 추출하기

# 자료형 변경
Building2['use_apr_day'] = Building2['use_apr_day'].astype(str) 

temp = []
for i in range(len(Building2)):
    temp.append(Building2.loc[i]['use_apr_day'][:4])

Building2['use_apr_day'] = temp

In [14]:
Building2.head()

,emd_nm,regstr_gb_cd_nm,etc_purpose_nm,use_apr_day
0,옥련동,일반,단독주택,1972
1,옥련동,일반,"단독주택, 제1,2종근린생활시설",2009
2,옥련동,일반,단독주택,1985
3,옥련동,일반,다가구주택,2009
4,옥련동,일반,"다가구주택, 근린생활시설",2000


In [15]:
Building2['etc_purpose_nm'].unique()

array(['단독주택', '단독주택, 제1,2종근린생활시설', '다가구주택', ...,
       '자동차관련시설(주차장),근린생활시설,판매시설', '판매시설,제1,2종근린생활시설,업무시설', '판매시설(대형점)'],
      dtype=object)

In [16]:
Building2['etc_purpose_nm'].value_counts()

주택                         1203
근린생활시설, 주택                  996
공동주택                        624
아파트                         562
단독주택                        485
                           ... 
공동주택(경비실-주출입구)                1
다가구 주택(17)                    1
공연장                           1
제1종,제2종근린생활시설,교육연구및복지시설       1
수리점,주차통로                      1
Name: etc_purpose_nm, Length: 1123, dtype: int64

In [17]:
Building3 = Building2.copy()

In [18]:
# 건축물 용도가 3개이상인 것 확인 !
# 개수가 별로 없으니까 삭제

a = []
for i in range(len(Building3)):
    name = Building3.loc[i]['etc_purpose_nm'].split(',')
    
    if len(name) >= 3:
        Building3.drop(i, axis = 0, inplace = True)
        a.append(name)

# 8천개중에 .. 331개 .. 삭제        
len(a)

331

In [19]:
Building3.reset_index(inplace = True)
Building3.drop('index', axis = 1, inplace = True)

In [20]:
# 주거용, 교육 및 사회용, 상업용 만 남기기

a = []

for i in range(len(Building3)):
    name = Building3.loc[i]['etc_purpose_nm'].split(',')
    
    for j in range(len(name)):
        
        if re.findall(r'주택',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '주거용'
        
        elif re.findall(r'다세대',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '주거용'

        elif re.findall(r'기숙사',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '주거용'
            
        elif re.findall(r'다가구',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '주거용'
            
        elif re.findall(r'아파트',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '주거용'
        
        elif re.findall(r'주  택',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '주거용'
            
        elif re.findall(r'연립',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '주거용'
            
        elif re.findall(r'오피스텔',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '주거용'
            
        elif re.findall(r'근',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '상업용'
            
        elif re.findall(r'고시원',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '주거용'
            
        elif re.findall(r'음식',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '상업용'
            
        elif re.findall(r'운수',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '상업용'

        elif re.findall(r'판매',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '상업용'
            
        elif re.findall(r'영업',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '상업용'

        elif re.findall(r'업무',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '상업용'
            
        elif re.findall(r'자동차',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '상업용'
            
        elif re.findall(r'위험물',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '상업용'
            
        elif re.findall(r'숙박',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '상업용'
            
        elif re.findall(r'게스트하우스',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '상업용'
            
        elif re.findall(r'사무',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '상업용'
            
        elif re.findall(r'생활',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '상업용'
            
        elif re.findall(r'카페',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '상업용'
            
        elif re.findall(r'매장',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '상업용'
            
        elif re.findall(r'소매점',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '상업용'

        elif re.findall(r'할인점',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '상업용'       
            
        elif re.findall(r'상가',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '상업용'

        elif re.findall(r'위락',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '상업용' 

        elif re.findall(r'교육',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
       
        elif re.findall(r'종교',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'

        elif re.findall(r'교회',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'보육',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'노유자',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'노인',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'휴게',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'의료',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'병동',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'문화',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'집회',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'사찰',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'사원',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'복지',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'독서',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'도서',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'운동',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'경로',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'

        elif re.findall(r'연구',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'휴양',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'학교',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'양노원',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'성당',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'어린이',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'유치원',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'새마을',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'트니스',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'체육관',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'학원',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'요양',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'커뮤니티',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        elif re.findall(r'관리',name[j]):
            Building3.loc[i,'etc_purpose_nm'] = '교육 및 사회용'
            
        else:
            a.append(i)

In [21]:
Building3 = Building3.drop(index = a)

In [22]:
Building3['etc_purpose_nm'].unique()

array(['주거용', '상업용', '교육 및 사회용'], dtype=object)

In [23]:
Building3.drop(columns = 'regstr_gb_cd_nm', inplace = True)

In [24]:
Building3.reset_index(inplace = True)

In [25]:
Building3 = Building3.drop(columns = 'index')

In [26]:
Building3.head()

,emd_nm,etc_purpose_nm,use_apr_day
0,옥련동,주거용,1972
1,옥련동,주거용,1985
2,옥련동,주거용,2009
3,옥련동,상업용,2000
4,옥련동,주거용,1985


In [27]:
Building3

,emd_nm,etc_purpose_nm,use_apr_day
0,옥련동,주거용,1972
1,옥련동,주거용,1985
2,옥련동,주거용,2009
3,옥련동,상업용,2000
4,옥련동,주거용,1985
...,...,...,...
7770,송도동,상업용,2020
7771,송도동,상업용,2009
7772,송도동,상업용,2009
7773,송도동,상업용,2017


# 최종 : 건축물대장_전체

In [28]:
# 저장
Building3.to_csv('건축물대장_전체.csv', encoding = 'utf-8', index = False)

In [29]:
# 불러오기
Building = pd.read_csv('건축물대장_전체.csv', encoding = 'utf-8')
Building.head()

,emd_nm,etc_purpose_nm,use_apr_day
0,옥련동,주거용,1972
1,옥련동,주거용,1985
2,옥련동,주거용,2009
3,옥련동,상업용,2000
4,옥련동,주거용,1985


# 최종 : 건축물대장_동별_용도별_노후건축물
- 2021년 기준

In [30]:
old_Building = Building3.copy()
old_Building['use_apr_day'] = old_Building['use_apr_day'].astype(int)
old_Building.head()

,emd_nm,etc_purpose_nm,use_apr_day
0,옥련동,주거용,1972
1,옥련동,주거용,1985
2,옥련동,주거용,2009
3,옥련동,상업용,2000
4,옥련동,주거용,1985


In [31]:
# 사용년도 : use_year
# 2021년 기준

old_Building['use_year'] = 2021 - old_Building['use_apr_day']

In [32]:
old_year = []

for i in old_Building['use_year']:
    if i >= 20:
        old_year.append(1)
        
    else:
        old_year.append(0)

In [33]:
old_Building['old_cnt'] = old_year

In [34]:
old_Building['purpose_cnt'] = 1

In [35]:
Building = old_Building.drop(columns = ['use_apr_day','use_year', 'use_apr_day'])

In [36]:
Building = Building.groupby(['etc_purpose_nm', 'emd_nm']).sum()
Building.reset_index(inplace = True)

In [37]:
Building.head()

,etc_purpose_nm,emd_nm,old_cnt,purpose_cnt
0,교육 및 사회용,동춘동,72,112
1,교육 및 사회용,선학동,23,38
2,교육 및 사회용,송도동,0,346
3,교육 및 사회용,연수동,51,79
4,교육 및 사회용,옥련동,76,109


In [38]:
# 저장
Building.to_csv('최종_건축물대장_용도_노후건축물.csv', encoding = 'utf-8', index = False)

In [39]:
# 불러오기
Building_fin = pd.read_csv('최종_건축물대장_용도_노후건축물.csv', encoding = 'utf-8')

In [40]:
Building_fin

,etc_purpose_nm,emd_nm,old_cnt,purpose_cnt
0,교육 및 사회용,동춘동,72,112
1,교육 및 사회용,선학동,23,38
2,교육 및 사회용,송도동,0,346
3,교육 및 사회용,연수동,51,79
4,교육 및 사회용,옥련동,76,109
5,교육 및 사회용,청학동,27,53
6,상업용,동춘동,187,326
7,상업용,선학동,79,133
8,상업용,송도동,0,299
9,상업용,연수동,184,245
